<ins>IPv4 Overview</ins>

An IPv4 address contains 32 bits. It is usually represented in
**dotted decimal quad notation** so it is easier to read and
communicate. Computers, of course, read this information in binary
form. And in order to calculate subnet masks, network addresses, and
broadcast addresses, the binary value must be known.

Converting a dotted decimal IP address to binary is simple. Given that there are 4 numbers,
each decimal number is represented in binary as an octet. The high order bits in binary are
always the leftmost ones.

###### Counting in Binary

In [1]:
headers = ('Binary','Powers of 2','Decimal')
xs = [(1,f"2^{x}",2**x) for x in range(8)]
print("".join([f"{headers[i]: <{12}}"+"".join([f"{x[i]: <{5}}" for x in xs[::-1]])+"\n" for i in range(3)]))

Binary      1    1    1    1    1    1    1    1    
Powers of 2 2^7  2^6  2^5  2^4  2^3  2^2  2^1  2^0  
Decimal     128  64   32   16   8    4    2    1    



The largest single value an IP address may have in decimal is 255
since this is the sum when all of the bits are added together, i.e.


In [4]:
print(" = ".join([" + ".join([f"{2**x}" for x in range(8)][::-1]), '%s' % sum([2**x for x in range(8)])]))

128 + 64 + 32 + 16 + 8 + 4 + 2 + 1 = 255


When all of the bits are turned off, the number is 0. The range of an
IP address is therefore 0 - 255, 256 values.

The following is how the IP address 68.65.16.250 would be converted
and represented in binary form.

In [5]:
def powers(n):
    ps = []
    for i in range(7,-1,-1):
        d = 2**i
        if n >= d:
            ps.append(i)
            n = n - d
    return ps

def factors(n):
    return [2**x for x in powers(n)]

ip_addr = '68.65.16.250'
col_label = ('', 'Decimal','Broken Down', 'Binary Addition', 'Binary')
row_label = ('First','Second','Third','Fourth')

rows = [(f"{row_label[i]} Octet", f"{n: <{9}}"," + ".join([str(f) for f in factors(n)])," + ".join([f"2^{p}" for p in powers(n)]),f"{n:08b}") for i,n in [(i,int(o)) for i,o in enumerate(ip_addr.split('.'))]]
ws = [max([len(r[c]) for r in rows]) for c in range(5)]

print("".join(["".join([f"{col: <{2+max(len(col),w)}}" for w,col in zip(ws,col_label)])+"\n", "".join(["".join([f"{r: <{2+w}}" for w,r in zip(ws,row)])+"\n" for row in rows])]))


              Decimal    Broken Down                 Binary Addition                    Binary    
First Octet   68         64 + 4                      2^6 + 2^2                          01000100  
Second Octet  65         64 + 1                      2^6 + 2^0                          01000001  
Third Octet   16         16                          2^4                                00010000  
Fourth Octet  250        128 + 64 + 32 + 16 + 8 + 2  2^7 + 2^6 + 2^5 + 2^4 + 2^3 + 2^1  11111010  



The end result looks like:


In [6]:
print("".join([f"{n:08b}" for n in [int(o) for o in ip_addr.split('.')]]))

01000100010000010001000011111010


### CIDR: Classless Inter-Domain Routing

CIDR networks are identified with a trailing "/" ("slash") and a
number that indicates how many bits are used to identify the network
portion of the address. A /24 would indicate that 24 bits are used to
identify the network and the remaining 8 bits are used to identify the
host.

#### CIDR Available Hosts

The formula to calculate the number of assignable addresses to CIDR
networks is similar to classful networking. Subtract the number of
network bits from 32. Raise 2 to that power and subtract 2 for the
network and broadcast addresses. For example, a /24 network has 2^(32 - 24) - 2
addresses available for the host assignment.


In [7]:
col_label = ('CIDR Notation', 'Host Formula   ', 'Available Hosts')
ws = [len(c) for c in col_label]
rows =  [(f"/{x}", f"2^(32 - {x: <{2}}) - 2", f"{(2 ** (32 - x) - 2):,}") for x in range(8,31)]
print("".join(["    ".join([f"{col: <{w}}" for w,col in zip(ws,col_label)])+'\n', "".join(["    ".join([f"{r: <{w}}" for w,r in zip(ws,row)])+'\n' for row in rows])]))


CIDR Notation    Host Formula       Available Hosts
/8               2^(32 - 8 ) - 2    16,777,214     
/9               2^(32 - 9 ) - 2    8,388,606      
/10              2^(32 - 10) - 2    4,194,302      
/11              2^(32 - 11) - 2    2,097,150      
/12              2^(32 - 12) - 2    1,048,574      
/13              2^(32 - 13) - 2    524,286        
/14              2^(32 - 14) - 2    262,142        
/15              2^(32 - 15) - 2    131,070        
/16              2^(32 - 16) - 2    65,534         
/17              2^(32 - 17) - 2    32,766         
/18              2^(32 - 18) - 2    16,382         
/19              2^(32 - 19) - 2    8,190          
/20              2^(32 - 20) - 2    4,094          
/21              2^(32 - 21) - 2    2,046          
/22              2^(32 - 22) - 2    1,022          
/23              2^(32 - 23) - 2    510            
/24              2^(32 - 24) - 2    254            
/25              2^(32 - 25) - 2    126            
/26         

As the table indicates, two /29 networks equals a /28 network. Two /28
networks equals a /27 network. Two /27 networks equals a /26
network. And so on, and so on. The notion of combining two smaller
networks into a larger one is another benefit of classless networks
named supernetting. In order to create a supernet the smaller networks
must be contiguous. For example, 192.0.2.240/29 and 192.0.2.248/29 can
form a supernet 192.0.2.240/28, but 192.0.2.240/29 and 192.0.2.8/29
could not.


<ins>The following illustrates how many /21 networks can fit into a /17 network</ins>

> * Subtract the network bits from 32.
>   * /17 = 32 - 17 and /21 = 32 - 21
> * Raise 2 to that power.
>   * 2^(32 - 17) and 2^(32 - 21)
> * Divide the larger networ by the smaller one.
>   * 2^(32 - 17) / 2^(32 - 21) = 2^15 / 2^11 = 2^(15 - 11) = 2^4 = 16


#### CIDR Networks

Since CIDR address spaces can overlap byte boundaries, the method to
tell which address is a part of which network is a little trickier
than with classful networking. Everything that needs to be known is
indicated by the "/" number, however. Given a network address
172.16.0.0/21, it is known that the first 21 bits of that address
represent the network portion. That leaves 11 bits for host
information, about 2,000 host addresses. To more easily see what that
range looks like, convert 172.16.0.0 into binary. In binary, the number looks like:

In [9]:
ip_addr = '172.16.0.0'
bit_string = "".join([f"{n:08b}" for n in [int(o) for o in ip_addr.split('.')]])
net_addr = bit_string[0:21]
host_addr = bit_string[21:]
print(f"{net_addr[0:8]}.{net_addr[8:16]}.{net_addr[16:]} {host_addr[0:len(host_addr)-8]}.{host_addr[-8:]}")

10101100.00010000.00000 000.00000000


The numbers before the break show the /21 network mask. No
modification can be done to the network portion of the address. The
remaining 11 bits are available for host assignment on the network.
After the break, the remaining 3 bits in the third octet can be added up
for a maximum value of decimal 7 (2^2 + 2^1 + 2^0). All of the bits in the
last octet are available and when converted to decimal equal 255. With all
of the bits turned on (all 1's), the decimal number turns out to be 172.16.7.255.
This is the end range of the 172.16/21 network, 172.16.0.0 - 172.16.7.255.


#### CIDR Subnet Mask

The process to determine the subnet mask for a CIDR address is
straightforward. The number of bits in the network portion of the
address are converted to 1's and right padded with 0's until there are
32 digits.  The sequence of digits is then divided into 4 octets
containing 8 digits each. From then, it is a matter of converting the
4 octets from binary to decimal.


In [10]:
def subnet_bitmask(num_bits):
    m = num_bits * '1' + (32 - num_bits) * '0'
    return ".".join([m[0:8], m[8:16], m[16:24], m[24:32]])

def bin_to_dec(bits):
    d = 0
    for bi in range(7,-1,-1):
        if bits[bi] == '1':
            d = d + 2**(7 - bi)
    return d

def subnet_mask(num_bits):
    m = subnet_bitmask(num_bits)
    return ".".join([str(bin_to_dec(octets)) for octets in m.split(".")])


col_label = ('CIDR Notation', "Convert to 1's and Right Pad       ", 'Subnet Mask')
ws = [len(c) for c in col_label]

rows = [(f"/{x}", f"{subnet_bitmask(x)}", f"{subnet_mask(x)}") for x in range(8,31)]
print("\n".join(["".join(["  ".join([f"{col: <{w}}" for w,col in zip(ws,col_label)])]), "".join(["  ".join([f"{r: <{w}}" for w,r in zip(ws,row)])+'\n' for row in rows])]))


CIDR Notation  Convert to 1's and Right Pad         Subnet Mask
/8             11111111.00000000.00000000.00000000  255.0.0.0  
/9             11111111.10000000.00000000.00000000  255.128.0.0
/10            11111111.11000000.00000000.00000000  255.192.0.0
/11            11111111.11100000.00000000.00000000  255.224.0.0
/12            11111111.11110000.00000000.00000000  255.240.0.0
/13            11111111.11111000.00000000.00000000  255.248.0.0
/14            11111111.11111100.00000000.00000000  255.252.0.0
/15            11111111.11111110.00000000.00000000  255.254.0.0
/16            11111111.11111111.00000000.00000000  255.255.0.0
/17            11111111.11111111.10000000.00000000  255.255.128.0
/18            11111111.11111111.11000000.00000000  255.255.192.0
/19            11111111.11111111.11100000.00000000  255.255.224.0
/20            11111111.11111111.11110000.00000000  255.255.240.0
/21            11111111.11111111.11111000.00000000  255.255.248.0
/22            11111111.111111